In [3]:
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
import keyring
from openai import OpenAI

client = OpenAI(api_key=keyring.get_password('openai', 'key_for_windows'))
model='gpt-3.5-turbo'

doc_list = [
    "우리나라는 2022년 코로나가 유행했다.",
    "우리나라 2024년 GDP 전망은 3.0%이다.",
    "우리나라는 2022년 국내총생산 중 연구개발 예산은 약 5%이다.", 
    "삼성전자 2024년 1분기 매출액은 약 7조원으로 잠정 추정됩니다.",
    "2024년 7월 19일 삼성전자 주가는 64,500원입니다."
]


bm25_retriever = BM25Retriever.from_texts(
    doc_list, metadatas=[{"source":1}]*len(doc_list)
)
bm25_retriever.k = 1

embedding = OpenAIEmbeddings(api_key=keyring.get_password('openai', 'key_for_windows'))
chroma_vectorstore = Chroma.from_texts(
    doc_list, embedding, metadatas=[{'source':1}] * len(doc_list)
)
chroma_retriever = chroma_vectorstore.as_retriever(search_kwarg={'k':1})
faiss_vectorstore = FAISS.from_texts(
    doc_list, embedding, metadatas=[{'source':1}] * len(doc_list)
)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={'k':1})

ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, chroma_retriever, faiss_retriever],
    weights=[0.2, 0.4, 0.4]
)

In [6]:
def search(query):
    ensemble_dcos = ensemble_retriever.invoke(query)
    return ensemble_dcos

def chatgpt_generate(query):
    
    response = client.chat.completions.create(
        model=model,
        messages=[{
            'role':'system',
            'content':'You are a helpful assistant.'
        }, {
            'role':'user',
            'content':query
        }]
    )
    
    answer = response.choices[0].message.content
    return answer

def get_answer(query, docs):
    prompt = f"""아래 질문을 기반으로 검색된 문서를 참고하여 질문에 대한 답변을 생성하시오.

    # 질문 : {query}
    """
    
    for i in range(len(docs)):
        prompt += f"# 문서{i+1}:" + docs[i]
    print(prompt)
    answer = chatgpt_generate(prompt)
    return answer

In [7]:
query = "삼성전자의 올해 매출액은?"
retrieved_docs = [doc.page_content for doc in search(query)]
answer = get_answer(query, retrieved_docs)
print(answer)

아래 질문을 기반으로 검색된 문서를 참고하여 질문에 대한 답변을 생성하시오.

    # 질문 : 삼성전자의 올해 매출액은?
    # 문서1:삼성전자 2024년 1분기 매출액은 약 7조원으로 잠정 추정됩니다.# 문서2:2024년 7월 19일 삼성전자 주가는 64,500원입니다.
죄송합니다. 제가 제공할 수 있는 정보는 2024년 1분기에 대한 삼성전자의 잠정 추정 매출액인 약 7조원입니다. 현재 올해 매출액에 대한 정확한 정보는 제공되지 않았습니다.
